Now we will continue on the [Conversation AI](https://conversationai.github.io/) dataset seen in [week 4 homework and lab](https://github.com/MIDS-scaling-up/v2/tree/master/week04). 
We shall use a version of pytorch BERT for classifying comments found at [https://github.com/huggingface/pytorch-pretrained-BERT](https://github.com/huggingface/pytorch-pretrained-BERT).  

The original implementation of BERT is optimised for TPU. Google released some amazing performance improvements on TPU over GPU, for example, see [here](https://medium.com/@ranko.mosic/googles-bert-nlp-5b2bb1236d78) - *BERT relies on massive compute for pre-training ( 4 days on 4 to 16 Cloud TPUs; pre-training on 8 GPUs would take 40–70 days).*. In response, Nvidia released [apex](https://devblogs.nvidia.com/apex-pytorch-easy-mixed-precision-training/), which gave mixed precision training. Weights are stored in float32 format, but calculations, like forward and backward propagation happen in float16 - this allows these calculations to be made with a [4X speed up](https://github.com/huggingface/pytorch-pretrained-BERT/issues/149).  

We shall apply BERT to the problem for classifiying toxicity, using apex from Nvidia. We shall compare the impact of hardware by running the model on a V100 and P100 and comparing the speed and accuracy in both cases.   

This script relies heavily on an existing [Kaggle kernel](https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila) from [yuval r](https://www.kaggle.com/yuval6967). 
  
*Disclaimer: the dataset used contains text that may be considered profane, vulgar, or offensive.*

In [1]:
# Download the training and the test corpus
!wget -nv --show-progress -O data/test.csv.zip https://www.dropbox.com/s/xp6bo8yo1vbv5yg/test.csv.zip?dl=1
!wget -nv --show-progress -O data/train.csv.zip https://www.dropbox.com/s/xei6z41mfrcnxcd/train.csv.zip?dl=1
# Download the pretrained weights for bert base. 
!wget -nv --show-progress -O data/uncased_L-12_H-768_A-12.zip \
        https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!wget -nv --show-progress  -O data/cased_L-12_H-768_A-12.zip \
        https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
# unzip weights & conifg and remove the original zip
!unzip -d data/ data/cased_L-12_H-768_A-12.zip && rm data/cased_L-12_H-768_A-12.zip
!unzip -d data/ data/uncased_L-12_H-768_A-12.zip && rm data/uncased_L-12_H-768_A-12.zip

data/test.csv.zip   100%[===================>]  12.09M  1.83MB/s    in 6.7s    
2020-10-19 03:29:16 URL:https://uc021b14464d725c73e598c8517e.dl.dropboxusercontent.com/cd/0/get/BBjxHPyj6OFlS0k97oH9qMZ561MVnb7_ESD5Bf_XhVsIrNa0pTXZNfdzx1uLDIWmDd6n0shZkesBiCAp8L6PBi712LRcl7b8omGseHDi25cQrCjpgOALO-rThV4lgDiMK1k/file?dl=1 [12673607/12673607] -> "data/test.csv.zip" [1]
data/train.csv.zip  100%[===================>] 272.99M  44.7MB/s    in 6.6s    
2020-10-19 03:29:24 URL:https://uc4bd2593ae64a8e8f9351e1058a.dl.dropboxusercontent.com/cd/0/get/BBi795H_sax5G5Zd0bFUNsWLgWOH4DhwMp8c9u-16AlW0zDMUIZa7iE6horgKQE1E-dlwhCeTQNrCpatACYNhM6zCn2uL_8NB0UqbjqknkG3oDi-y9IgIgj9G9H-QlpduZo/file?dl=1 [286248352/286248352] -> "data/train.csv.zip" [1]
data/uncased_L-12_H 100%[===================>] 388.84M  2.91MB/s    in 2m 15s  
2020-10-19 03:31:40 URL:https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip [407727028/407727028] -> "data/uncased_L-12_H-768_A-12.zip" [1]
data/cased_L-12_H

In [18]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys, os
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from sklearn import metrics

%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Let's activate CUDA for GPU based operations
device=torch.device('cuda')

Change the PATH variable to whereever your `week06/hw` directory is located.  
**For the final run we would like you to have a train_size of at least 1 Million rows, and a valid size of at least 500K rows. When you first run the script, feel free to work with a reduced train and valid size for speed.** 

In [4]:
# In bert we need all inputs to have the same length, we will use the first 220 characters. 
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
# We shall run a single epoch (ie. one pass over the data)
EPOCHS = 1
PATH = '.'#'/root/v2/week06/hw' # /root/v2/week06/hw"
DATA_DIR = os.path.join(PATH, "data")
WORK_DIR = os.path.join(PATH, "workingdir")

# Validation and training sizes are here. 
train_size= 1000000 
valid_size= 500000

This should be the files you downloaded earlier when you ran `download.sh`

In [5]:
os.listdir(DATA_DIR)

['train.csv.zip',
 'cased_L-12_H-768_A-12.zip',
 'test.csv.zip',
 'uncased_L-12_H-768_A-12.zip',
 'uncased_L-12_H-768_A-12',
 'cased_L-12_H-768_A-12',
 'tmp']

We shall install pytorch BERT implementation.   
If you would like to experiment with or view any code (purely optional, and not graded :) ), you can copy the files from the repo https://github.com/huggingface/pytorch-pretrained-BERT  

In [6]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
%%capture
from transformers import BertModel, BertConfig, BertTokenizer, BertForSequenceClassification
from transformers import AdamW as BertAdam

We shall now load the model. When you run this, comment out the `capture` command to understand the archecture.

In [8]:
%%capture
bert_config = BertConfig()

Bert needs a special formatting of sentences, so we have a sentence start and end token, as well as separators.   
Thanks to this [script](https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming) for a fast convertor of the sentences.

In [9]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

Now we load the BERT tokenizer and convert the sentences.

In [10]:
%%time
# tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_all = pd.read_csv(os.path.join(DATA_DIR, "train.csv.zip")).sample(train_size+valid_size,random_state=SEED)
print('loaded %d records' % len(train_all))

# Make sure all comment_text values are strings
train_all['comment_text'] = train_all['comment_text'].astype(str) 

sequences = convert_lines(train_all["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
train_all=train_all.fillna(0)

loaded 1500000 records


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



33722
CPU times: user 27min 19s, sys: 9.15 s, total: 27min 28s
Wall time: 27min 17s


Let us look at how the tokenising works in BERT, see below how it recongizes misspellings - words the model never saw. 

In [11]:
train_all[["comment_text", 'target']].head()

,comment_text,target
458232,It's difficult for many old people to keep up ...,0.000000
272766,She recognized that her tiny-handed husband is...,0.166667
339129,"HPHY76,\nGood for you for thinking out loud, w...",0.000000
773565,And I bet that in the day you expected your Je...,0.500000
476233,Kennedy will add a much needed and scientifica...,0.000000


Lets tokenize some text (I intentionally mispelled some words to check berts subword information handling)

In [12]:
text = 'Hi, I am learning new things in w251 about deep learning the cloud and teh edge.'
tokens = tokenizer.tokenize(text)
' '.join(tokens)

'hi , i am learning new things in w ##25 ##1 about deep learning the cloud and te ##h edge .'

Added start and end token and convert to ids. This is how it is fed into BERT.

In [13]:
tokens = ["[CLS]"] + tokens + ["[SEP]"]
input_ids = tokenizer.convert_tokens_to_ids(tokens)
' '.join(map(str, input_ids))

'101 7632 1010 1045 2572 4083 2047 2477 1999 1059 17788 2487 2055 2784 4083 1996 6112 1998 8915 2232 3341 1012 102'

When BERT converts this sentence to a torch tensor below is shape of the stored tensors.  
We have 12 input tensors, while the sentence tokens has length 23; where are can you see the 23 tokens in the tensors ?... **Feel free to post in slack or discuss in class**

In [14]:
# put input on gpu and make prediction
bert = BertForSequenceClassification.from_pretrained('bert-base-uncased').cuda()
# bert = BertModel.from_pretrained(WORK_DIR).cuda()
bert_output = bert(torch.tensor([input_ids]).cuda())

print('Sentence tokens {}'.format(tokens))
print('Number of tokens {}'.format(len(tokens)))
print('Tensor shapes : {}'.format([b.cpu().detach().numpy().shape for b in bert_output[0]]))
print('Number of torch tensors : {}'.format(len(bert_output[0])))

Sentence tokens ['[CLS]', 'hi', ',', 'i', 'am', 'learning', 'new', 'things', 'in', 'w', '##25', '##1', 'about', 'deep', 'learning', 'the', 'cloud', 'and', 'te', '##h', 'edge', '.', '[SEP]']
Number of tokens 23
Tensor shapes : [(2,)]
Number of torch tensors : 1


As it is a binary problem, we change our target to [0,1], instead of float.   
We also split the dataset into a training and validation set, 

In [15]:
train_all['target']=(train_all['target']>=0.5).astype(float)
# Training data - sentences
X = sequences[:train_size] 
# Target - the toxicity. 
y = train_all[['target']].values[:train_size]
X_val = sequences[train_size:]                
y_val = train_all[['target']].values[train_size:]

In [16]:
test_df=train_all.tail(valid_size).copy()
train_df=train_all.head(train_size)

**From here on in we would like you to run BERT.**   
**Please do rely on the script available -  [Kaggle kernel](https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila) from [yuval r](https://www.kaggle.com/yuval6967) - for at least the first few steps up to training and prediction.**  
    
**Note, instead of using the apex in yuval's code, we would like you to use the new native [pytorch apex version](https://pytorch.org/docs/stable/amp.html).** 
**You can see some examples of native [apex](https://pytorch.org/docs/stable/notes/amp_examples.html#typical-mixed-precision-training) here.**


**1)**   
**Load the training set to a training dataset. For this you need to load the X sequences and y objects to torch tensors**   
**You can use `torch.utils.data.TensorDataset` to input these into a train_dataset.**

In [17]:
# Training data creations
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))

**2)**  
**Set your learning rate and batch size; and optionally random seeds if you want reproducable results**   
**Load your pretrained BERT using `BertForSequenceClassification`**   
**Initialise the gradients and place the model on cuda, set up your optimiser and decay parameters**
**note, we will not use apex like yuval. So no `amp.initialise` needed.**

In [18]:
SEED = 1234
lr=2e-5
batch_size = 32
# batch_size=4
accumulation_steps = 1
# accumulation_steps = 8
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [19]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',cache_dir=None,num_labels=1)
model.zero_grad()
model = model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]


**3)**  
**Start training your model by iterating through batches in a single epoch of the data**    
**I copied the new amp process shown [here](https://github.com/aws/deep-learning-containers/blob/b9ccd5928fe2ecd8aafc4c1660b39fe265ed879f/test/dlc_tests/container_tests/bin/pytorch_tests/testPyTorchAMP#L31-L40) and it worked out.**   
A few tips on the changes needed,   
1) You will need to use `torch.cuda.amp.autocast()` and `torch.cuda.amp.GradScaler()` like seen in the reference code. 
2) From yuval's code, the prediction and loss calculation must be executed with autocast (see new amp process code).      
3) Loss scaling, the omptimiser step and update can be taken from the new amp code, but please do the optimiser step and update with accumulation from yuval's code.     
4) For some reason, yuval's code, in the `y_pred` line, returns a tuple when run with autocast. Just index into the first element to get the prediction.    

In [20]:
train = train_dataset
tq = tqdm_notebook(range(EPOCHS))
num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = BertAdam(optimizer_grouped_parameters, lr=lr)


lossf=None
loss_fn=nn.MSELoss()
scaler = torch.cuda.amp.GradScaler()
for epochz in tq:
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0
    avg_accuracy = 0
    tk0= tqdm_notebook(enumerate(train_loader), total=len(train_loader),leave=False)
    for i,(x_batch, y_batch) in tk0:
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)[0]
            loss = F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        scaler.scale(loss).backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()

        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)



/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


**4)**  
**Store your trained model to disk, you will need it if you choose section 8C.**

In [21]:
torch.save(model.state_dict(), 'bert_pytorch.bin')

**5)**   
**Now make a prediction for your validation set.**  

In [22]:
%%capture 
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=1).cuda()
model.load_state_dict(torch.load('bert_pytorch.bin'))
model.to(device)
for param in model.parameters():
    param.requires_grad=False
model.eval()  # print the model ---> you can remove 
val_batch_size = 128
valid_preds = np.zeros((len(X_val)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=val_batch_size, shuffle=False)
tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,)  in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)[0]
    valid_preds[i*val_batch_size:(i+1)*val_batch_size]=pred[:,0].detach().cpu().squeeze().numpy()
    if i % 20 == 0:
        print("batch: ", i, " of ", len(valid_loader))

**6)**  
**In the yuval's kernel he get a metric based on the metric for the jigsaw competition - it is quite complicated. Instead, we would like you to measure the `AUC`, similar to how you did in homework 04. You can compare the results to HW04**  
*A tip, if your score is lower than homework 04 something is wrong....*

In [23]:
# From baseline kernel

def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]>0.5
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]>0.5]
    return compute_auc((subgroup_examples[label]>0.5), subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]>0.5) & (df[label]<=0.5)]
    non_subgroup_positive_examples = df[(df[subgroup]<=0.5) & (df[label]>0.5)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]>0.5) & (df[label]>0.5)]
    non_subgroup_negative_examples = df[(df[subgroup]<=0.5) & (df[label]<=0.5)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]>0.5])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

In [24]:
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['target']

train_df = train_df.drop(['comment_text'],axis=1)
# convert target to 0,1
train_df['target']=(train_df['target']>=0.5).astype(float)

In [25]:
MODEL_NAME = 'model'
test_df[MODEL_NAME]=torch.sigmoid(torch.tensor(valid_preds)).numpy()
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
2,homosexual_gay_or_lesbian,2842,0.837389,0.844927,0.970364
6,black,3870,0.849242,0.798789,0.981189
7,white,6589,0.851772,0.826757,0.977377
5,muslim,5447,0.868134,0.861129,0.972994
4,jewish,2031,0.896256,0.905660,0.965465
1,female,14140,0.921953,0.937268,0.960798
0,male,11311,0.925049,0.929091,0.966965
3,christian,9916,0.931840,0.950499,0.955615
8,psychiatric_or_mental_illness,1145,0.933079,0.927052,0.971130


0.9248018232833495

**7)**  
**Can you show/print the validation sentences predicted with the highest and lowest toxicity ?**

AUC score : {:.5f}


### **Remember to terminate your instance at the end.**